# SERVE/RETURN ANALYSIS

In [1]:
import pandas as pd
import numpy as np
import re
from dataclasses import dataclass
from typing import Optional, List, Tuple


In [2]:
pts = pd.read_csv("DADOS_COMPLETOS.csv", dtype=str)
matches = pd.read_csv("charting-m-matches.csv", usecols=["match_id","Surface","Round"])
pts = pts.merge(matches, on="match_id", how="left")


In [3]:

# ───── 1. CONSTANTS & DATACLASS ─────
@dataclass
class Shot:
    hitter: str
    stroke: str
    pos_mod: Optional[str] = None
    net_cord: bool = False
    sector: Optional[str] = None
    err: str = ""
    out: str = ""

STROKE_LABEL = {
    "f": "Forehand",  "b": "Backhand",
    "s": "Backhand slice",  "r": "Forehand slice",
    "v": "Forehand volley", "z": "Backhand volley",
    "o": "Forehand overhead", "p": "Backhand overhead",
    "u": "Forehand drop shot", "y": "Backhand drop shot",
    "l": "Forehand lob", "m": "Backhand lob",
    "h": "Forehand half-volley", "i": "Backhand half-volley",
    "j": "Forehand swinging volley", "k": "Backhand swinging volley",
    "t": "Tweener / trick shot", "q": "Unknown stroke",
    "S": "Serve"
}
SERVE_DIR = {"4": "Wide", "5": "Body", "6": "Down-the-T", "0": "Unknown"}
ERR_LABEL = {"n": "net", "w": "wide", "d": "deep", "x": "wide_deep",
             "g": "foot fault", "!": "shank", "e": "unknown", "": ""}
OUTCOME_LABEL = {"*": "Winner", "#": "Forced error",
                 "@": "Unforced error", "": ""}
SPECIAL_CODES = {
    'S': 'Point to Server', 'R': 'Point to Returner',
    'P': 'Point Penalty on Server','Q': 'Point Penalty on Returner',
    'C': 'Incorrect Challenge',  'V': 'Time Violation on Serve'
}

In [4]:
# ───── 2. ROBUST PARSER ─────
def parse_point(code: str) -> Tuple[Optional[str], List[Shot]]:
    code = str(code).strip()
    if not code:
        return None, []
    if code in SPECIAL_CODES:
        return None, [Shot("System", code, err=SPECIAL_CODES[code])]
    for sc, lbl in SPECIAL_CODES.items():
        if code.endswith(sc):
            return None, [Shot("System", sc, err=lbl)]

    code = code.lstrip('c')  # strip lets

    # serve token
    dir_code, serve_pos_mod, rest = None, None, code
    if code and code[0] in SERVE_DIR:
        dir_code, rest = code[0], code[1:]
        if rest.startswith("+"):
            serve_pos_mod, rest = "+", rest[1:]

    serve_err, serve_out = "", ""
    if dir_code is None and len(rest) == 1 and rest in ERR_LABEL:
        serve_err, rest = rest, ""
    elif rest and len(rest) == 1:
        if rest[0] in ERR_LABEL:
            serve_err, rest = rest[0], ""
        elif rest[0] in OUTCOME_LABEL:
            serve_out, rest = rest[0], ""

    shots: List[Shot] = [Shot("srv", "S", serve_pos_mod, False,
                              dir_code, serve_err, serve_out)]

    # rally tokens
    server_turn, idx = False, 0
    while idx < len(rest):
        if rest[idx] not in STROKE_LABEL:
            break
        stroke = rest[idx]
        next_idx = len(rest)
        for i in range(idx+1, len(rest)):
            if rest[i] in STROKE_LABEL:
                next_idx = i
                break

        token = rest[idx+1:next_idx]
        modifiers = re.findall(r'[;\+\-\=\^]', token)
        digits    = "".join(re.findall(r'\d', token))
        err       = "".join(re.findall(r'[nwdxg!e]', token))
        outcome   = "".join(re.findall(r'[\*@#]', token))

        shots.append(Shot(
            hitter="srv" if server_turn else "ret",
            stroke=stroke,
            pos_mod=next((c for c in modifiers if c in "+-=^"), None),
            net_cord=';' in modifiers,
            sector=digits or None,
            err=err.lower(),
            out=outcome
        ))

        idx = next_idx
        server_turn = not server_turn
        if outcome:
            break

    return dir_code, shots

In [5]:
# ---------------------------------------------------------
# H1  Court-side detector (non-tiebreak + tiebreak logic)
# ---------------------------------------------------------
POINT_MAP = {"0":0, "15":1, "30":2, "40":3, "AD":4}

def point_side(row):
    """
    Return 'deuce' or 'ad' for each point in pts:
     - If in a tiebreak (TbSet and 6-6 games), Pts is 'x-y' numeric.
     - Otherwise Pts is tennis numeric ('0','15','30','40','AD').
    """
    # Are we inside the tiebreak game?
    in_tb = row["TbSet"] and row["Gm1"] == "6" and row["Gm2"] == "6"
    p_str = row["Pts"].strip()

    if in_tb:
        # e.g. '5-3'
        p1, p2 = map(int, p_str.split("-"))
        return "deuce" if (p1 + p2) % 2 == 0 else "ad"

    # normal game, server-first scoring
    left, right = p_str.split("-")
    s1 = POINT_MAP[left]
    s2 = POINT_MAP[right]
    return "deuce" if (s1 + s2) % 2 == 0 else "ad"



# ---------------------------------------------------------
# H2  Tiny lexer for ONE serve attempt
# ---------------------------------------------------------
def lex_serve(token:str):
    """
    Return (dir_code, err_code, out_code).
    Works for the short tokens found in `1st` or `2nd`.
    """
    token = str(token).strip()
    if not token:
        return None, "", ""

    dir_code, err_code, out_code = None, "", ""
    if token[0] in SERVE_DIR:           # 4/5/6/0
        dir_code, token = token[0], token[1:]

    # exactly one trailing symbol at most
    if token and token[0] in ERR_LABEL:
        err_code = token[0]
    elif token and token[0] in OUTCOME_LABEL:
        out_code = token[0]

    return dir_code, err_code, out_code


In [6]:
# ───── 3. FLAG LOGIC ─────
def make_flags(row):
    shots = row["shots"]
    if not shots:
        return pd.Series({"is_ace": False,
                          "is_unret": False,
                          "is_sp1": False})

    ace = shots[0].out == '*'

    unret = (
        shots[0].out == '#' or
        (len(shots) == 2 and shots[-1].hitter == "ret" and shots[-1].out == "#")
    )

    sp1 = (
        len(shots) == 3 and                     # *** exactly three ***
        shots[1].hitter == "ret" and shots[1].out == "" and
        shots[2].hitter == "srv" and shots[2].out == "*"
    )

    return pd.Series({"is_ace": ace,
                      "is_unret": unret,
                      "is_sp1": sp1})


In [7]:
def tag_return(row):
    sh = row.shots
    if len(sh) < 2:      # ace or unreturned
        return pd.Series(dict(
            rtn_played  = False,
            rtn_winner  = False,
            rtn_forced  = False,
            rtn_unforce = False,
            rtn_dir     = None
        ))
    rtn = sh[1]
    return pd.Series(dict(
        rtn_played  = True,
        rtn_winner  = rtn.out == '*',
        rtn_forced  = rtn.out == '#',
        rtn_unforce = rtn.out == '@',
        rtn_dir     = rtn.sector      # raw 1-9; map later if desired
    ))




In [8]:
# ───── 4. MAIN PIPELINE ─────

# 1) normalize booleans
to_bool = lambda v: str(v).strip().upper() in {"1","TRUE"}
pts["TbSet"] = pts["TbSet"].apply(to_bool)
pts["Svr"]   = pts["Svr"].apply(to_bool)

# 2) identify the server & returner columns
pts[["player1","player2"]] = pts["match_id"].apply(
    lambda m: pd.Series([p.replace("_"," ") for p in str(m).split("-")[-2:]])
)
pts["server_name"] = pts.apply(
    lambda r: r.player1 if r.Svr else r.player2, axis=1
)

# 3) extract raw rally code
def extract_code(v):
    s = str(v).strip()
    return "" if s.lower()=="nan" or not s else s

pts["raw_code"] = pts.apply(
    lambda r: extract_code(r["2nd"]) or extract_code(r["1st"]),
    axis=1
)

# 4) parse the full shots list
pts["shots"] = pts["raw_code"].map(lambda c: parse_point(c)[1])

# 5) compute serve flags (ace/unret/sp1)
pts = pd.concat([pts, pts.apply(make_flags, axis=1)], axis=1)

# 6) compute return flags (now that shots exists)
pts = pd.concat([pts, pts.apply(tag_return, axis=1)], axis=1)

In [9]:
def safe_point_side(row):
    # split the Pts string
    left, right = row["Pts"].split("-")
    # convert each side: use POINT_MAP if present, else int()
    try:
        p1 = POINT_MAP[left]
    except KeyError:
        p1 = int(left)
    try:
        p2 = POINT_MAP[right]
    except KeyError:
        p2 = int(right)
    # parity rule covers both regular games and tiebreakers
    return "deuce" if (p1 + p2) % 2 == 0 else "ad"


In [10]:
# 7) call the two new helpers
# normalize the tiebreak-flag column (TB?)
#pts["TB?"] = pts["TB?"].apply(lambda v: str(v).strip().upper() in {"1","TRUE"})

pts["side"] = pts.apply(safe_point_side, axis=1)

pts[["dir1","err1","out1"]] = pts["1st"].map(lex_serve).tolist()
pts[["dir2","err2","out2"]] = pts["2nd"].map(lex_serve).tolist()

# ─── 3-A explode into serve_df ──────────────────────────
records = []
for _, r in pts.iterrows():
    # now r.server_name, r.shots, r.is_sp1, r.side, r.dir1… all exist
    side    = r.side
    surface = r.Surface
    round_  = r.Round

    # first serve
    d1,e1,o1 = r.dir1, r.err1, r.out1
    records.append({
        "match_id":  r.match_id,
        "server":    r.server_name,
        "returner":  (r.player2 if r.server_name==r.player1 else r.player1),
        "serve_num": 1,
        "side":      side,
        "surface":   surface,
        "round":     round_,
        "dir_code":  d1,
        "err_code":  e1,
        "out_code":  o1,
        "ace":       (o1=="*"),
        "unret":     (o1=="#"),
        "sp_plus1":  r.is_sp1
    })

    # second serve if first was a fault
    if e1:
        d2,e2,o2 = r.dir2, r.err2, r.out2
        records.append({
            **records[-1],
            "serve_num": 2,
            "dir_code":  d2,
            "err_code":  e2,
            "out_code":  o2,
            "ace":       (o2=="*"),
            "unret":     (o2=="#"),
            "sp_plus1":  False
        })

serve_df = pd.DataFrame(records)
# ────────────────────────────────────────────────────────


In [11]:
serve_df.to_csv("serve_stats.csv", index=False)

# SERVE EFFICIENCY 

This Serve Efficiency counts _every_ serve, even the ones not in.

In [12]:
# 1.1 Aggregate per server
serve_stats = (
    serve_df
      .groupby("server")
      .agg(
        total_serves = ("serve_num", "size"),
        total_aces   = ("ace",       "sum"),
        total_unret  = ("unret",     "sum"),
        total_sp1    = ("sp_plus1",  "sum")
      )
)

# 1.2 Compute rates + composite Serve Efficiency
serve_stats = serve_stats.assign(
    ace_rate    = 100 * serve_stats.total_aces   / serve_stats.total_serves,
    unret_rate  = 100 * serve_stats.total_unret  / serve_stats.total_serves,
    sp1_rate    = 100 * serve_stats.total_sp1    / serve_stats.total_serves,
)
serve_stats["serve_eff"] = (
    serve_stats.ace_rate
  + serve_stats.unret_rate
  + serve_stats.sp1_rate
).round(2)

serve_stats = serve_stats.sort_values("serve_eff", ascending=False)
serve_stats.head(10)


,total_serves,total_aces,total_unret,total_sp1,ace_rate,unret_rate,sp1_rate,serve_eff
server,,,,,,,,
Giovanni Mpetshi Perricard,974,175,80,61,17.967146,8.213552,6.262834,32.44
Luca Vanni,127,11,18,5,8.661417,14.173228,3.937008,26.77
Johannes Haerteis,123,25,0,7,20.325203,0.000000,5.691057,26.02
Ante Pavic,109,16,6,6,14.678899,5.504587,5.504587,25.69
Reilly Opelka,5037,804,158,320,15.961882,3.136788,6.352988,25.45
Hamad Medjedovic,202,21,18,10,10.396040,8.910891,4.950495,24.26
John Isner,9677,1410,200,696,14.570631,2.066756,7.192312,23.83
Nicolas Jarry,4378,308,403,320,7.035176,9.205116,7.309274,23.55
Nick Kyrgios,12028,1507,561,682,12.529099,4.664117,5.670103,22.86


Organizando tabela

In [13]:
# Re-sort the full stats by Serve Efficiency (descending):
serve_stats_sorted = serve_stats.sort_values("serve_eff", ascending=False)
serve_stats_sorted.head(10)   # just to confirm


,total_serves,total_aces,total_unret,total_sp1,ace_rate,unret_rate,sp1_rate,serve_eff
server,,,,,,,,
Giovanni Mpetshi Perricard,974,175,80,61,17.967146,8.213552,6.262834,32.44
Luca Vanni,127,11,18,5,8.661417,14.173228,3.937008,26.77
Johannes Haerteis,123,25,0,7,20.325203,0.000000,5.691057,26.02
Ante Pavic,109,16,6,6,14.678899,5.504587,5.504587,25.69
Reilly Opelka,5037,804,158,320,15.961882,3.136788,6.352988,25.45
Hamad Medjedovic,202,21,18,10,10.396040,8.910891,4.950495,24.26
John Isner,9677,1410,200,696,14.570631,2.066756,7.192312,23.83
Nicolas Jarry,4378,308,403,320,7.035176,9.205116,7.309274,23.55
Nick Kyrgios,12028,1507,561,682,12.529099,4.664117,5.670103,22.86


 Filtering by number of serves

In [14]:
# 1) To print the entire table:
#serve_stats

# 2) To filter to only those with, say, at least 1000 serves:
threshold = 1000
filtered = serve_stats[serve_stats["total_serves"] >= threshold]
filtered

# 3) (Optional) Sort the filtered table by Serve Efficiency:
filtered.sort_values("serve_eff", ascending=False)


,total_serves,total_aces,total_unret,total_sp1,ace_rate,unret_rate,sp1_rate,serve_eff
server,,,,,,,,
Reilly Opelka,5037,804,158,320,15.961882,3.136788,6.352988,25.45
John Isner,9677,1410,200,696,14.570631,2.066756,7.192312,23.83
Nicolas Jarry,4378,308,403,320,7.035176,9.205116,7.309274,23.55
Nick Kyrgios,12028,1507,561,682,12.529099,4.664117,5.670103,22.86
Quentin Halys,1037,122,48,64,11.764706,4.628737,6.171649,22.57
...,...,...,...,...,...,...,...,...
Diego Schwartzman,4808,69,27,138,1.435108,0.561564,2.870216,4.87
Andrei Chesnokov,1611,42,11,25,2.607076,0.682806,1.551831,4.84
Mats Wilander,3970,61,0,126,1.536524,0.000000,3.173804,4.71


Pesquisando múltiplos jogadores

In [15]:
players = ["Roger Federer", "Rafael Nadal", "Novak Djokovic", "Joao Fonseca"]

# pick those rows…
subset = serve_stats_sorted.loc[players]

# …then sort by Serve Efficiency descending:
subset_sorted = subset.sort_values("serve_eff", ascending=False)

subset_sorted



,total_serves,total_aces,total_unret,total_sp1,ace_rate,unret_rate,sp1_rate,serve_eff
server,,,,,,,,
Roger Federer,70432,4895,477,4396,6.949966,0.677249,6.241481,13.87
Novak Djokovic,53641,2664,470,2422,4.966350,0.876195,4.515203,10.36
Joao Fonseca,844,38,3,41,4.502370,0.355450,4.857820,9.72
Rafael Nadal,42193,1299,191,1825,3.078710,0.452682,4.325362,7.86


Slice of a player in the ranking

In [16]:
# 1) Define your threshold
threshold = 1000

# 2) Filter the fully-sorted table
filtered = serve_stats_sorted[serve_stats_sorted["total_serves"] >= threshold]

# 3) Now pick your player and window
player = "Ivo Karlovic"
window = 2

# 4) Locate his position _within_ that filtered table
pos = filtered.index.get_loc(player)

# 5) Compute slice boundaries
start = max(pos - window, 0)
end   = pos + window + 1

# 6) Show player ± window
filtered.iloc[start:end]


,total_serves,total_aces,total_unret,total_sp1,ace_rate,unret_rate,sp1_rate,serve_eff
server,,,,,,,,
Milos Raonic,8566,1032,221,594,12.047630,2.579967,6.934392,21.56
Christopher Eubanks,1521,92,76,127,6.048652,4.996713,8.349770,19.40
Ivo Karlovic,3870,406,48,283,10.490956,1.240310,7.312661,19.04
Jiri Lehecka,3803,236,257,230,6.205627,6.757823,6.047857,19.01
Arthur Fils,3878,239,292,194,6.162971,7.529654,5.002579,18.70


# RETURNER EFFICIENCY

In [17]:
# 2.1  server × returner table
pair = (
    serve_df
      .groupby(["server", "returner"])
      .agg(
          plays = ("serve_num",  "size"),
          ace   = ("ace",        "sum"),
          unret = ("unret",      "sum"),
          sp1   = ("sp_plus1",   "sum"),
      )
      .assign(
          effpts  = lambda d: d.ace + d.unret + d.sp1,
          SE_pair = lambda d: 100 * d.effpts / d.plays
      )
      #.drop(columns=["ace", "unret", "sp1"])          # optional
)

return_threshold = 1000
pair = pair[pair.plays >= return_threshold]


# 2.2  server-level baseline
base = (
    serve_df
      .groupby("server")
      .agg(
          plays = ("serve_num",  "size"),
          ace   = ("ace",        "sum"),
          unret = ("unret",      "sum"),
          sp1   = ("sp_plus1",   "sum"),
      )
      .assign(
          effpts  = lambda d: d.ace + d.unret + d.sp1,
          SE_base = lambda d: 100 * d.effpts / d.plays
      )
      .loc[:, ["SE_base"]]                          # keep just the one column
)

# 2.3  drop-off table
pair = pair.join(base, on="server")
pair["DSE"] = pair.SE_base - pair.SE_pair


In [18]:
# rebuild the per-returner table
rtn_eff = (
    pts.query("rtn_played")
       .groupby("player2")                # player2 = returner
       .agg(
           returns   = ("rtn_played", "size"),
           winners   = ("rtn_winner", "sum"),
           unforced  = ("rtn_unforce", "sum"),
       )
       .assign(
           win_rate = lambda d: 100 * d.winners / d.returns,
           err_rate = lambda d: 100 * d.unforced / d.returns   # ← only Unforced!
       )
       .query("returns >= 10000")          # ≥ ALTER HERE FOR FILTER BY NUMBER OF RETURNS
       .sort_values(["win_rate", "err_rate"], ascending=[False, True])
)

print(rtn_eff.head(20).sort_values("err_rate", ascending=True))


                       returns  winners  unforced  win_rate  err_rate
player2                                                              
Novak Djokovic           17156      226       489  1.317323  2.850315
Rafael Nadal             32575      449      1075  1.378358  3.300077
Andy Murray              19244      319       669  1.657660  3.476408
Juan Martin Del Potro    13789      195       493  1.414171  3.575314
Alexander Zverev         15252      126       569  0.826121  3.730658
Daniil Medvedev          23441      262       900  1.117700  3.839427
Dominic Thiem            15082      148       659  0.981302  4.369447
Ivan Lendl               15579      549       704  3.523975  4.518904
Lleyton Hewitt           12299      274       557  2.227823  4.528823
Jannik Sinner            14013      162       652  1.156069  4.652822
Roger Federer            30304      565      1451  1.864440  4.788147
Andre Agassi             19520      659       981  3.376025  5.025615
Carlos Alcaraz      

In [19]:
return_rank = (
    pair.groupby("returner")
        .agg(
            total_returns = ("plays", "sum"),
            DSE   = ("DSE", "mean")
        )
        .query("total_returns >= 10000")      # ≥ ALTER HERE FOR FILTER BY NUMBER OF RETURNS
        .sort_values("DSE", ascending=False)
)


New Indicator: Drop in Serve Efficiency

In [20]:
return_rank

,total_returns,DSE
returner,,
Andy Murray,11726,2.334420
Novak Djokovic,26506,2.111835
Roger Federer,36462,2.051559
Stefan Edberg,10367,1.274231
Rafael Nadal,17123,0.379144
Pete Sampras,10997,-1.258081


In [21]:
# per-server baseline SE already in 'base'
# compute *overall* average SE across all servers
field_avg_SE = base.SE_base.mean()
print(f"Average Serve Efficiency across field: {field_avg_SE:.2f}%")


Average Serve Efficiency across field: 9.10%


In [22]:
return_rank = (
    pair.groupby("returner")
        .agg(
            total_returns = ("plays", "sum"),
            mean_DSE  = ("DSE", "mean")
        )
        .assign(
            lift_vs_field = lambda d: d.mean_DSE - (field_avg_SE - d.mean_DSE)
            # essentially: drop vs each server compared to what *average* returners allow
        )
        .query("total_returns >= 10000")
        .sort_values("lift_vs_field", ascending=False)
)


In [23]:
top_servers = (
    serve_stats
    .query("total_serves >= 3000")
    .head(50)
    .index
)
top_servers

Index(['Reilly Opelka', 'John Isner', 'Nicolas Jarry', 'Nick Kyrgios',
       'Milos Raonic', 'Ivo Karlovic', 'Jiri Lehecka', 'Arthur Fils',
       'Goran Ivanisevic', 'Kevin Anderson', 'Alexander Bublik',
       'Matteo Berrettini', 'Andrey Rublev', 'Taylor Fritz',
       'Denis Shapovalov', 'Richard Krajicek', 'Ben Shelton', 'Ugo Humbert',
       'Felix Auger Aliassime', 'Jan Lennard Struff', 'Benoit Paire',
       'Maxime Cressy', 'Alexander Zverev', 'Andy Roddick', 'Hubert Hurkacz',
       'Pete Sampras', 'Marin Cilic', 'Karen Khachanov', 'Stefanos Tsitsipas',
       'Tallon Griekspoor', 'Daniil Medvedev', 'Roger Federer',
       'Jo Wilfried Tsonga', 'Sebastian Korda', 'Holger Rune',
       'Grigor Dimitrov', 'Casper Ruud', 'Juan Martin Del Potro',
       'Laslo Djere', 'Tommy Paul', 'Lorenzo Sonego', 'Aslan Karatsev',
       'Patrick Rafter', 'Feliciano Lopez', 'Michael Stich', 'Marat Safin',
       'Fernando Gonzalez', 'Stan Wawrinka', 'Gael Monfils', 'Boris Becker'],
      dtyp

In [24]:
elite_pair = pair[pair.index.get_level_values("server").isin(top_servers)]
elite_rank = (
    elite_pair.groupby("returner")
              .agg(
                  returns_vs_elite = ("plays", "sum"),
                  mean_DSE  = ("DSE", "mean")
              )
              .query("returns_vs_elite >= 5000")
              .sort_values("mean_DSE", ascending=False)
)
print(elite_rank.head(20))




                returns_vs_elite  mean_DSE
returner                                  
Novak Djokovic             12993  2.540088
Roger Federer              11603  2.392485
Rafael Nadal                6822  1.738358
Stefan Edberg               6554  1.552613
Andre Agassi                7624 -1.845228


# RETURN DIRECTION

In [25]:
# Cell 1 – RETURN DIRECTION  
# This cell computes the return direction and other return stats

if "err1" not in pts.columns:
    pts[["dir1","err1","out1"]] = pts["1st"].map(lex_serve).tolist()
    pts[["dir2","err2","out2"]] = pts["2nd"].map(lex_serve).tolist()
return_rows = []
for _, r in pts.iterrows():

    # skip aces / unreturned serves
    if not r.rtn_played:
        continue

    rtn = r.shots[1]                     # first shot by returner

    # ------- split the two-digit sector code -------
    sec = rtn.sector or ""               # "", "8", "17", …
    lat = sec[0] if len(sec) >= 1 else None      # 1-6  (lateral)
    dep = sec[1] if len(sec) >= 2 else None      # 7/8/9 (depth)
    # ----------------------------------------------

    # first-serve return unless the first serve carried an error code
    serve_num = 1 if r.err1 == "" else 2

    return_rows.append({
        "match_id" : r.match_id,
        "returner" : r.player2 if r.server_name == r.player1 else r.player1,
        "server"   : r.server_name,
        "serve_num": serve_num,
        "side"     : r.side,
        "surface"  : r.Surface,
        "round"    : r.Round,
        "lat_sec"  : lat,
        "dep_sec"  : dep,
        "winner"   : rtn.out == "*",
        "unforced" : rtn.out == "@",
    })

return_df = pd.DataFrame(return_rows)


In [26]:
# Cell 2 – RETURN DIRECTION (label the two axes)

LATERAL_MAP = {
    "1": "fh_corner",      # right-hander’s FH / lefty BH
    "2": "middle",
    "3": "bh_corner",
    "4": "wide",
    "5": "body",
    "6": "T"
}

DEPTH_MAP = {
    "7": "shallow",
    "8": "deep",
    "9": "very_deep"
}

return_df["lat_bin"]   = return_df.lat_sec.map(LATERAL_MAP).fillna("unk")
return_df["depth_bin"] = return_df.dep_sec.map(DEPTH_MAP).fillna("unk")


In [27]:
# Cell 3 – RETURN DIRECTION (% table with both dimensions)
depth_order = ["shallow", "deep", "very_deep", "unk"]

rtn_dir_pct = (
    return_df
      .groupby([
          "returner", "serve_num", "side", "surface",
          "lat_bin", "depth_bin"              # ← two-level column index
      ])
      .size()
      .groupby(level=[0,1,2,3])               # total per (player,serve,side,surface)
      .apply(lambda s: 100*s/s.sum())         # convert to %
      .unstack(fill_value=0)
      .reindex(depth_order, axis = 1)                                # columns = lat × depth
)

# example slice: Novak Djokovic, vs 1st serves, deuce side, all surfaces
rtn_dir_pct.xs(("Novak Djokovic", 1, "deuce"), level=[0,1,2])


depth_bin                                                  shallow       deep  \
surface returner       serve_num side  surface lat_bin                          
Clay    Novak Djokovic 1         deuce Clay    bh_corner  2.907370   5.949966   
                                               fh_corner  3.211629   9.533469   
                                               middle     7.674104  22.887086   
                                               unk        0.000000   0.000000   
Grass   Novak Djokovic 1         deuce Grass   bh_corner  3.810119   4.622111   
                                               fh_corner  4.434728   7.245472   
                                               middle     8.619613  19.675203   
                                               unk        0.000000   0.000000   
Hard    Novak Djokovic 1         deuce Hard    bh_corner  3.132498   4.905359   
                                               fh_corner  3.852306   8.211144   
                                               middle     9.130898  21.154359   
                                               unk        0.000000   0.000000   

depth_bin                                                 very_deep        unk  
surface returner       serve_num side  surface lat_bin                          
Clay    Novak Djokovic 1         deuce Clay    bh_corner   3.617309   3.651116  
                                               fh_corner   5.070994   4.901961  
                                               middle     12.576065  12.204192  
                                               unk         0.000000   5.814740  
Grass   Novak Djokovic 1         deuce Grass   bh_corner   3.060587   5.559026  
                                               fh_corner   3.810119   7.058089  
                                               middle     13.304185  14.428482  
                                               unk         0.000000   4.372267  
Hard    Novak Djokovic 1         deuce Hard    bh_corner   2.519328   4.758731  
                                               fh_corner   4.372167   6.238336  
                                               middle     13.703013  12.969875  
                                               unk         0.000000   5.051986

In [28]:
'''# 1) raw counts by lat & depth
dj_all = (
    return_df
      .query("returner == 'Novak Djokovic'")
      .groupby(["lat_bin", "depth_bin"])
      .size()
)
# just to inspect:
print(dj_all.head())
# bh_corner  deep         4374
#            shallow      2081
#            unk          2127
#            very_deep    1985
# ...

# 2) divide by the grand total, then ×100
total = dj_all.sum()
print("Total returns:", total)   # sanity check

pct_global = (dj_all / total) * 100

# 3) pivot into a DataFrame and reorder
pct_global = (
    pct_global
      .unstack(fill_value=0)
      .reindex(depth_order, axis=1)
)

# now you should see floats that add up to 100 across the *entire* table:
print(pct_global)
print("Grand sum:", pct_global.values.sum())  # should be 100.0
  




SyntaxError: incomplete input (2334571900.py, line 1)

In [29]:
# Cell 4 – RETURN DIRECTION (player-specific % table)
player_return_query = (
    return_df.query("returner == 'Novak Djokovic'")
             .groupby(["lat_bin", "depth_bin"])
             .size()
)

pct_global = 100 * player_return_query / player_return_query.sum()        # ONE denominator
pct_global = (
    pct_global.unstack(fill_value=0)
              .reindex(depth_order, axis=1)     # keep column order
)

pct_global.index = (
    pd.MultiIndex.from_product(
        [["Novak Djokovic"], pct_global.index],   # top = player, 2nd = lat_bin
        names=["player", "lat_bin"]
    )
)

pct_global


depth_bin                  shallow       deep  very_deep        unk
player         lat_bin                                             
Novak Djokovic bh_corner  5.679740  11.938099   5.417724   5.805289
               fh_corner  2.278992   5.652447   3.316139   3.993013
               middle     7.492017  20.726548  12.953410  10.709899
               unk        0.000000   0.000000   0.000000   4.036682

Full table

In [30]:
rtn_dir_pct = rtn_dir_pct.reindex(depth_order, axis=1, level=1)
rtn_dir_pct

depth_bin                                                                                      shallow  \
returner         serve_num side  surface returner         serve_num side  surface lat_bin                
Aaron Krickstein 1         ad    Clay    Aaron Krickstein 1         ad    Clay    bh_corner   8.695652   
                                                                                  fh_corner   0.000000   
                                                                                  middle      4.347826   
                                                                                  unk         0.000000   
                                 Hard    Aaron Krickstein 1         ad    Hard    bh_corner  13.114754   
...                                                                                                ...   
Zsombor Piros    2         deuce Clay    Zsombor Piros    2         deuce Clay    fh_corner   0.000000   
                                                                                  middle      7.692308   
                                 Hard    Zsombor Piros    2         deuce Hard    bh_corner   0.000000   
                                                                                  fh_corner   0.000000   
                                                                                  middle      0.000000   

depth_bin                                                                                         deep  \
returner         serve_num side  surface returner         serve_num side  surface lat_bin                
Aaron Krickstein 1         ad    Clay    Aaron Krickstein 1         ad    Clay    bh_corner  30.434783   
                                                                                  fh_corner   4.347826   
                                                                                  middle     30.434783   
                                                                                  unk         0.000000   
                                 Hard    Aaron Krickstein 1         ad    Hard    bh_corner   8.196721   
...                                                                                                ...   
Zsombor Piros    2         deuce Clay    Zsombor Piros    2         deuce Clay    fh_corner   7.692308   
                                                                                  middle     23.076923   
                                 Hard    Zsombor Piros    2         deuce Hard    bh_corner  12.500000   
                                                                                  fh_corner  12.500000   
                                                                                  middle     25.000000   

depth_bin                                                                                    very_deep  \
returner         serve_num side  surface returner         serve_num side  surface lat_bin                
Aaron Krickstein 1         ad    Clay    Aaron Krickstein 1         ad    Clay    bh_corner   4.347826   
                                                                                  fh_corner   4.347826   
                                                                                  middle      0.000000   
                                                                                  unk         0.000000   
                                 Hard    Aaron Krickstein 1         ad    Hard    bh_corner   0.546448   
...                                                                                                ...   
Zsombor Piros    2         deuce Clay    Zsombor Piros    2         deuce Clay    fh_corner   0.000000   
                                                                                  middle     15.384615   
                                 Hard    Zsombor Piros    2         deuce Hard    bh_corner   0.000000   
                                                                            

RANKING VERY-DEEP RETURNS

In [31]:
import pandas as pd

# ─── PARAMETERS ───
MIN_RETURNS = 10000     # only keep players with ≥ this many returns
DEPTH_FILTER = "very_deep"    # which depth to rank on
LAT_ORDER = ["bh_corner","fh_corner","middle","unk"]

# ─── 1. Totals per player ───
tot = (
    return_df
      .groupby("returner")
      .size()
      .rename("total_returns")
)

# ─── 2. Subset to the target depth ───
vd = return_df[return_df.depth_bin == DEPTH_FILTER]

# 2a) count of very_deep returns per player
tot_vd = vd.groupby("returner").size().rename(f"{DEPTH_FILTER}_count")

# 2b) breakdown of those very_deep by lat_bin
vd_lat = (
    vd.groupby(["returner","lat_bin"])
      .size()
      .unstack(fill_value=0)
      .reindex(columns=LAT_ORDER, fill_value=0)
)

# ─── 3. Assemble into one DataFrame ───
rank_df = pd.concat([tot, tot_vd, vd_lat], axis=1).fillna(0)

# ─── 4. Compute percentages of ALL returns ───
rank_df[f"{DEPTH_FILTER}_pct"] = 100 * rank_df[f"{DEPTH_FILTER}_count"] / rank_df["total_returns"]
for lat in LAT_ORDER:
    rank_df[f"{DEPTH_FILTER}_{lat}_pct"] = 100 * rank_df[lat] / rank_df["total_returns"]

# ─── 5. Filter by minimum returns ───
rank_df = rank_df[rank_df["total_returns"] >= MIN_RETURNS]

# ─── 6. Sort by the metric you care about ───
#    e.g. percent of *all* returns that were very_deep+middle:
rank_df = rank_df.sort_values(f"{DEPTH_FILTER}_middle_pct", ascending=False)

# ─── 7. Select & display columns ───
cols = [
    "total_returns",
    f"{DEPTH_FILTER}_count", f"{DEPTH_FILTER}_pct"
] + [f"{DEPTH_FILTER}_{lat}_pct" for lat in LAT_ORDER]

print(rank_df[cols].head(20))


                    total_returns  very_deep_count  very_deep_pct  \
returner                                                            
Hubert Hurkacz              20773           5418.0      26.081933   
Novak Djokovic              36639           7946.0      21.687273   
Dominic Thiem               12270           2751.0      22.420538   
Jannik Sinner               13853           2657.0      19.179961   
Stefanos Tsitsipas          11709           2232.0      19.062260   
Grigor Dimitrov             10152           1887.0      18.587470   
Alexander Zverev            11331           1904.0      16.803460   
Daniil Medvedev             16985           2915.0      17.162202   
Andy Murray                 18672           3194.0      17.105827   
Carlos Alcaraz              11407           1962.0      17.199965   
Roger Federer               49027           7499.0      15.295653   
Rafael Nadal                29632           4429.0      14.946679   
Andre Agassi                14230 